In [1]:
'''بِسْمِ اللهِ الرَّحْمٰنِ الرَّحِيْمِ'''

'بِسْمِ اللهِ الرَّحْمٰنِ الرَّحِيْمِ'

In [2]:
'''سُبْحَـٰنَكَ لَا عِلْمَ لَنَآ إِلَّا مَا عَلَّمْتَنَآ ۖ إِنَّكَ أَنتَ الْعَلِيمُ الْحَكِيمُ'''

'سُبْحَـٰنَكَ لَا عِلْمَ لَنَآ إِلَّا مَا عَلَّمْتَنَآ ۖ إِنَّكَ أَنتَ الْعَلِيمُ الْحَكِيمُ'

In [3]:
from IPython.core.display import HTML
HTML(r"""
<style>
    * {
        font-family: Monaco;
    }
</style>
""")

In [4]:
import os,sys,warnings,re,math,gc,time
warnings.filterwarnings("ignore")
os.cpu_count()

12

In [5]:
os.environ["POLARS_MAX_THREADS"] = "20"
import numpy as np
import pandas as pd
import polars as pl
import polars.selectors as cs
from glob import glob
from pathlib import Path
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
from plotly.subplots import make_subplots
pio.templates["mod"] = go.layout.Template(layout=dict(font=dict(family="Monaco",size=15)))
pio.templates.default = "mod+plotly_dark"
from sklearn.metrics import auc,roc_auc_score
from sklearn.model_selection import train_test_split
import lightgbm as lgb
%xmode Minimal

Exception reporting mode: Minimal


In [6]:
pl.Config.set_fmt_str_lengths(100)
pl.Config.set_streaming_chunk_size(20000)
pl.Config.set_thousands_separator(",")
pl.Config.set_float_precision(2)
pl.Config.set_fmt_float("full")
pl.Config.set_tbl_rows(40)

polars.config.Config

In [7]:
path_to_train = Path("Downloads/credit_risk/train/")
path_to_test = Path("Downloads/credit_risk/test/")
path_to_features = Path("Downloads/feature_definitions.csv")

In [8]:
feat_df = pl.read_csv(path_to_features)
feat_df.head()

Variable,Description
str,str
"""actualdpd_943P""","""Days Past Due (DPD) of previous contract (actual)."""
"""actualdpdtolerance_344P""","""DPD of client with tolerance."""
"""addres_district_368M""","""District of the person's address."""
"""addres_role_871L""","""Role of person's address."""
"""addres_zip_823M""","""Zip code of the address."""


In [9]:
all_train_files = glob("Downloads/credit_risk/train/*.parquet")
all_test_files = glob("Downloads/credit_risk/test/*.parquet")
train_files_df = pl.DataFrame({"index":range(len(all_train_files)),"path":all_train_files})
test_files_df = pl.DataFrame({"index":range(len(all_test_files)),"path":all_test_files})
train_files_df = (
    train_files_df
    .with_columns(
        (pl.col("path").str.split("/").list.get(-1)).alias("filename")
    )
    .sort(by="filename")
)
test_files_df = (
    test_files_df
    .with_columns(
        (pl.col("path").str.split("/").list.get(-1)).alias("filename")
    )
    .sort(by="filename")
)
display(train_files_df.head())
display(test_files_df.head())

index,path,filename
i64,str,str
31,"""Downloads/credit_risk/train/train_applprev_1_0.parquet""","""train_applprev_1_0.parquet"""
13,"""Downloads/credit_risk/train/train_applprev_1_1.parquet""","""train_applprev_1_1.parquet"""
0,"""Downloads/credit_risk/train/train_applprev_2.parquet""","""train_applprev_2.parquet"""
7,"""Downloads/credit_risk/train/train_base.parquet""","""train_base.parquet"""
11,"""Downloads/credit_risk/train/train_credit_bureau_a_1_0.parquet""","""train_credit_bureau_a_1_0.parquet"""


index,path,filename
i64,str,str
31,"""Downloads/credit_risk/test/test_applprev_1_0.parquet""","""test_applprev_1_0.parquet"""
35,"""Downloads/credit_risk/test/test_applprev_1_1.parquet""","""test_applprev_1_1.parquet"""
9,"""Downloads/credit_risk/test/test_applprev_1_2.parquet""","""test_applprev_1_2.parquet"""
1,"""Downloads/credit_risk/test/test_applprev_2.parquet""","""test_applprev_2.parquet"""
33,"""Downloads/credit_risk/test/test_base.parquet""","""test_base.parquet"""


In [10]:
display(train_files_df.filter(pl.col("filename").str.contains("base")))
_ind = train_files_df.filter(pl.col("filename").str.contains("base")).select("index").item()

index,path,filename
i64,str,str
7,"""Downloads/credit_risk/train/train_base.parquet""","""train_base.parquet"""


In [11]:
train_base = (
    pl.scan_parquet(all_train_files[_ind])
    .select(
        pl.col("case_id").cast(pl.UInt32).alias("case_id_base"),
        cs.contains("date").str.to_date().alias("Date"),
        pl.col("target").cast(pl.UInt8)
    )
    .with_columns(
        pl.col("Date").dt.month().alias("month"),
        pl.col("Date").dt.week().alias("week"),
        pl.col("Date").dt.weekday().alias("weekday")
    )

    .select(~cs.contains("target"),cs.contains("target"))
)
display(train_base.fetch().head())
print(train_base.collect(streaming=True).shape)

case_id_base,Date,month,week,weekday,target
u32,date,i8,i8,i8,u8
0,2019-01-03,1,1,4,0
1,2019-01-03,1,1,4,0
2,2019-01-04,1,1,5,0
3,2019-01-03,1,1,4,0
4,2019-01-04,1,1,5,1


(1526659, 6)


In [12]:
display(test_files_df.filter(pl.col("filename").str.contains("base")))
_ind = test_files_df.filter(pl.col("filename").str.contains("base")).select("index").item()

index,path,filename
i64,str,str
33,"""Downloads/credit_risk/test/test_base.parquet""","""test_base.parquet"""


In [13]:
test_base = (
    pl.scan_parquet(all_test_files[_ind])
    .select(
        pl.col("case_id").cast(pl.UInt32).alias("case_id_base"),
        cs.contains("date").str.to_date().alias("Date"),
    )
    .with_columns(
        pl.col("Date").dt.month().alias("month"),
        pl.col("Date").dt.week().alias("week"),
        pl.col("Date").dt.weekday().alias("weekday"),
        pl.lit(0).cast(pl.UInt8).alias("target")
    )
)
display(test_base.fetch().head())
print(test_base.collect().shape)

case_id_base,Date,month,week,weekday,target
u32,date,i8,i8,i8,u8
"57,543",2020-10-06,10,41,2,0
"57,549",2020-10-06,10,41,2,0
"57,551",2020-10-06,10,41,2,0
"57,552",2020-10-07,10,41,3,0
"57,569",2020-10-06,10,41,2,0


(10, 6)


In [14]:
all_files_df = (
    pl.concat(
        [
            train_files_df,
            test_files_df
        ]
    )
    .sort(by="filename")
)
all_files_df.head()

index,path,filename
i64,str,str
31,"""Downloads/credit_risk/test/test_applprev_1_0.parquet""","""test_applprev_1_0.parquet"""
35,"""Downloads/credit_risk/test/test_applprev_1_1.parquet""","""test_applprev_1_1.parquet"""
9,"""Downloads/credit_risk/test/test_applprev_1_2.parquet""","""test_applprev_1_2.parquet"""
1,"""Downloads/credit_risk/test/test_applprev_2.parquet""","""test_applprev_2.parquet"""
33,"""Downloads/credit_risk/test/test_base.parquet""","""test_base.parquet"""


In [15]:
total_base = (
    pl.concat(
        [
            train_base,
            test_base
        ]
    )
)
total_base.fetch().head()

case_id_base,Date,month,week,weekday,target
u32,date,i8,i8,i8,u8
0,2019-01-03,1,1,4,0
1,2019-01-03,1,1,4,0
2,2019-01-04,1,1,5,0
3,2019-01-03,1,1,4,0
4,2019-01-04,1,1,5,1


In [16]:
def read_and_cast(filter_string:str):
    _files_list = all_files_df.filter(pl.col("filename").str.contains(filter_string))["path"].to_list()
    return (
        pl.concat(
            [
                pl.scan_parquet(_) for _ in _files_list
            ],
            how="vertical_relaxed"
        )
        .select(
            cs.by_name("case_id").cast(pl.UInt32),
            cs.contains("num_group").cast(pl.UInt16),
            cs.ends_with("D").cast(pl.Date),
            cs.ends_with("L","T","M").cast(pl.String).cast(pl.Categorical),
            cs.ends_with("P","A").cast(pl.Float32)
        )
    )

def select_impuatable(df:pl.LazyFrame,threshold=0.5):
    _ = df.collect(streaming=True)
    __ = _.shape[0]
    cols = (
        _
        .null_count()
        .pipe(lambda df: df/__)
        .transpose(include_header=True)
        .filter(pl.col("column_0") <= threshold)
        ["column"].to_list()
        
    )
    del _,__
    gc.collect()
    return df.select(cols)

def grouping(df:pl.LazyFrame):
    return (
        df
        .group_by("case_id")
        .agg(
            (cs.ends_with("P","A","D") | cs.contains("num_group")).max().prefix("max_"),
            (cs.ends_with("P","A","D") | cs.contains("num_group")).min().prefix("min_"),
            (cs.ends_with("P","A","D") | cs.contains("num_group")).mean().prefix("mean_"),
            cs.ends_with("L","T","M").mode().drop_nulls().first().prefix("first_"),
        )
    )

## Past Application

In [17]:
total_past_shallow = read_and_cast("applprev_1").pipe(select_impuatable)
total_past_shallow.fetch().head()

case_id,num_group1,approvaldate_319D,creationdate_885D,dateactivated_425D,firstnonzeroinstldate_307D,cancelreason_3545846M,credtype_587L,district_544M,education_1138M,familystate_726L,inittransactioncode_279L,isbidproduct_390L,pmtnum_8L,postype_4733339M,profession_152M,rejectreason_755M,rejectreasonclient_4145042M,status_219L,tenor_203L,actualdpd_943P,annuity_853A,credacc_credlmt_575A,credamount_590A,currdebt_94A,downpmt_134A,mainoccupationinc_437A,maxdpdtolerance_577P,outstandingdebt_522A
u32,u16,date,date,date,date,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,f32,f32,f32,f32,f32,f32,f32,f32,f32
"57,543",0,null,2018-07-10,null,2018-08-09,"""P94_109_143""","""COL""","""P197_47_166""","""a55475b1""",null,"""POS""","""false""","""31.0""","""P149_40_170""","""a55475b1""","""P99_56_166""","""P94_109_143""","""D""","""31.0""",0.00,"1,519.80",0.00,"37,998.00",null,0.00,"30,000.00",null,null
"57,549",0,null,2019-04-09,null,2019-05-10,"""P94_109_143""","""CAL""","""P7_110_89""","""P97_36_170""","""SINGLE""","""CASH""","""false""","""12.0""","""P67_102_161""","""a55475b1""","""P45_84_106""","""P94_109_143""","""D""","""12.0""",0.00,"2,048.80",0.00,"20,000.00",null,0.00,"12,000.00",null,null
"57,551",0,null,2019-11-23,null,2019-12-23,"""P94_109_143""","""CAL""","""a55475b1""","""a55475b1""",null,"""CASH""","""false""","""12.0""","""P67_102_161""","""a55475b1""","""P99_56_166""","""P94_109_143""","""D""","""12.0""",0.00,"1,871.80",0.00,"20,000.00",null,0.00,"22,000.00",null,null
"57,551",1,null,2018-08-08,null,2018-09-08,"""P94_109_143""","""CAL""","""P197_47_166""","""P97_36_170""","""MARRIED""","""CASH""","""false""","""18.0""","""P46_145_78""","""a55475b1""","""P45_84_106""","""P94_109_143""","""D""","""18.0""",0.00,"5,401.00",0.00,"60,000.00",null,0.00,"50,000.00",null,null
"57,552",0,null,2017-03-01,null,2017-03-29,"""P94_109_143""","""CAL""","""P11_36_178""","""P97_36_170""","""MARRIED""","""CASH""","""false""","""18.0""","""P46_145_78""","""a55475b1""","""P45_84_106""","""P94_109_143""","""D""","""18.0""",0.00,"4,834.20",0.00,"50,000.00",null,0.00,"16,000.00",null,null


In [18]:
total_past_depth = read_and_cast("applprev_2").pipe(select_impuatable)
total_past_depth.fetch().head()

case_id,num_group1,num_group2,cacccardblochreas_147M,conts_type_509L
u32,u16,u16,cat,cat
"57,543",0,0,null,"""PRIMARY_MOBILE"""
"57,549",0,2,null,"""PHONE"""
"57,549",0,0,null,"""PRIMARY_MOBILE"""
"57,549",0,1,null,"""HOME_PHONE"""
"57,551",1,1,null,"""PRIMARY_MOBILE"""


In [19]:
total_static_base = read_and_cast("static_0").pipe(select_impuatable)
total_static_base.fetch().head()

case_id,firstdatedue_489D,lastactivateddate_801D,lastapplicationdate_877D,lastapprdate_640D,applicationcnt_361L,applications30d_658L,applicationscnt_1086L,applicationscnt_464L,applicationscnt_629L,applicationscnt_867L,clientscnt12m_3712952L,clientscnt3m_3712950L,clientscnt6m_3712949L,clientscnt_100L,clientscnt_1022L,clientscnt_1071L,clientscnt_1130L,clientscnt_157L,clientscnt_257L,clientscnt_304L,clientscnt_360L,clientscnt_493L,clientscnt_533L,clientscnt_887L,clientscnt_946L,cntincpaycont9m_3716944L,cntpmts24_3658933L,commnoinclast6m_3546845L,credtype_322L,daysoverduetolerancedd_3976961L,deferredmnthsnum_166L,disbursementtype_67L,eir_270L,homephncnt_628L,inittransactioncode_186L,interestrate_311L,…,sellerplacecnt_915L,sellerplacescnt_216L,twobodfilling_608L,actualdpdtolerance_344P,amtinstpaidbefduel24m_4187115A,annuity_780A,annuitynextmonth_57A,avgdbddpdlast24m_3658932P,avgdpdtolclosure24_3658938P,avginstallast24m_3658937A,avgmaxdpdlast9m_3716943P,credamount_770A,currdebt_22A,currdebtcredtyperange_828A,disbursedcredamount_1113A,downpmt_116A,lastapprcredamount_781A,maininc_215A,maxannuity_159A,maxdbddpdtollast12m_3658940P,maxdebt4_972A,maxdpdfrom6mto36m_3546853P,maxdpdlast12m_727P,maxdpdlast24m_143P,maxdpdlast3m_392P,maxdpdlast6m_474P,maxdpdlast9m_1059P,maxdpdtolerance_374P,maxinstallast24m_3658928A,mindbddpdlast24m_3658935P,posfpd10lastmonth_333P,posfpd30lastmonth_3976960P,posfstqpd30lastmonth_3976962P,price_1097A,sumoutstandtotal_3546847A,totaldebt_9A,totalsettled_863A
u32,date,date,date,date,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,…,cat,cat,cat,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32
"57,543",null,null,2018-07-10,null,"""0.0""","""0.0""","""0.0""","""0.0""","""0.0""","""1.0""","""0.0""","""0.0""","""0.0""","""0.0""","""0.0""","""0.0""","""0.0""","""0.0""","""0.0""","""0.0""","""0.0""","""0.0""","""0.0""","""0.0""","""1.0""",null,null,"""0.0""","""CAL""",null,"""0.0""","""GBA""","""0.45""","""0.0""","""CASH""","""0.45""",…,"""0.0""","""1.0""","""FO""",null,null,"7,637.20",0.00,null,null,null,null,"100,000.00",0.00,0.00,"100,000.00",0.00,null,null,0.00,null,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,null,null,0.00,0.00,0.00,0.00,null,0.00,0.00
"57,549",null,null,2019-04-09,null,"""0.0""","""0.0""","""0.0""","""0.0""","""0.0""","""0.0""","""0.0""","""0.0""","""0.0""","""0.0""","""0.0""","""0.0""","""0.0""","""1.0""","""0.0""","""0.0""","""0.0""","""0.0""","""0.0""","""0.0""","""0.0""",null,null,"""0.0""","""CAL""",null,"""0.0""","""GBA""","""0.15""","""1.0""","""CASH""","""0.15""",…,"""0.0""","""0.0""","""BO""",null,null,902.60,0.00,null,null,null,null,"10,000.00",0.00,0.00,"10,000.00",0.00,null,null,0.00,null,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,null,null,0.00,0.00,1.00,null,null,0.00,0.00
"57,551",null,null,2019-11-23,null,"""0.0""","""0.0""","""0.0""","""0.0""","""0.0""","""2.0""","""0.0""","""0.0""","""0.0""","""0.0""","""0.0""","""0.0""","""0.0""","""1.0""","""0.0""","""0.0""","""0.0""","""0.0""","""0.0""","""0.0""","""0.0""",null,null,"""0.0""","""CAL""",null,"""0.0""","""GBA""","""0.15""","""0.0""","""CASH""","""0.15""",…,"""0.0""","""1.0""","""BO""",null,null,"3,610.20",0.00,null,null,null,null,"40,000.00",0.00,0.00,"40,000.00",0.00,null,null,0.00,null,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,null,null,0.00,0.00,0.00,null,null,0.00,0.00
"57,552",null,null,2017-03-01,null,"""0.0""","""0.0""","""0.0""","""0.0""","""0.0""","""1.0""","""0.0""","""0.0""","""0.0""","""0.0""","""0.0""","""0.0""","""1.0""","""0.0""","""0.0""","""0.0""","""0.0""","""0.0""","""1.0""","""0.0""","""0.0""",null,null,"""0.0""","""CAL""",null,"""0.0""","""GBA""","""0.45""","""0.0""","""CASH""","""0.45""",…,"""0.0""","""0.0""","""FO""",null,null,"6,964.40",0.00,null,null,null,null,"90,000.00",0.00,0.00,"90,000.00",0.00,null,null,0.00,null,0.00,0.00,0.00,0.00,0.00,0.00,0.00

In [20]:
total_person_shallow = read_and_cast("person_1").pipe(select_impuatable)
total_person_shallow.fetch().head()

case_id,num_group1,birth_259D,contaddr_district_15M,contaddr_matchlist_1032L,contaddr_smempladdr_334L,contaddr_zipcode_807M,education_927M,empladdr_district_926M,empladdr_zipcode_114M,incometype_1044T,language1_981M,personindex_1023L,persontype_1072L,persontype_792L,registaddr_district_1083M,registaddr_zipcode_184M,role_1084L,safeguarantyflag_411L,sex_738L,type_25L,mainoccupationinc_384A
u32,u16,date,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,f32
"57,543",0,1996-08-01,"""P107_30_170""","""false""","""false""","""P144_138_111""","""P97_36_170""","""P16_81_96""","""P144_138_111""","""EMPLOYED""","""P209_127_106""","""0.0""","""1.0""","""1.0""","""P107_30_170""","""P144_138_111""","""CL""","""false""","""M""","""PRIMARY_MOBILE""","36,000.00"
"57,543",1,null,"""a55475b1""",null,null,"""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""",null,"""a55475b1""","""1.0""","""1.0""","""5.0""","""a55475b1""","""a55475b1""","""EM""",null,null,"""PHONE""",null
"57,543",2,null,"""a55475b1""",null,null,"""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""",null,"""a55475b1""",null,"""5.0""",null,"""a55475b1""","""a55475b1""","""PE""",null,null,"""PHONE""",null
"57,549",0,1992-03-01,"""P7_110_89""","""false""","""false""","""P166_45_172""","""P97_36_170""","""P7_110_89""","""P166_45_172""","""EMPLOYED""","""P10_39_147""","""0.0""","""1.0""","""1.0""","""P7_110_89""","""P166_45_172""","""EM""","""false""","""M""","""PRIMARY_MOBILE""","15,000.00"
"57,549",1,null,"""a55475b1""",null,null,"""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""",null,"""a55475b1""","""1.0""","""1.0""","""5.0""","""a55475b1""","""a55475b1""","""EM""",null,null,"""ALTERNATIVE_PHONE""",null


In [21]:
total_person_depth = read_and_cast("person_2").pipe(select_impuatable)
total_person_depth.fetch().head()

case_id,num_group1,num_group2,addres_district_368M,addres_zip_823M,conts_role_79M,empls_economicalst_849M,empls_employer_name_740M
u32,u16,u16,cat,cat,cat,cat,cat
"57,552",0,0,"""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1"""
"57,569",0,0,"""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1"""
"57,630",0,0,"""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1"""
"57,631",0,0,"""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1"""
"57,632",0,0,"""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1"""


In [22]:
total_other_shallow = read_and_cast("other_1").pipe(select_impuatable)
total_other_shallow.fetch().head()

case_id,num_group1,amtdebitincoming_4809443A,amtdebitoutgoing_4809440A,amtdepositbalance_4809441A,amtdepositincoming_4809444A,amtdepositoutgoing_4809442A
u32,u16,f32,f32,f32,f32,f32
"57,635",0,0.00,0.00,"12,028.80",0.00,"4,011.00"
"57,694",0,0.00,0.00,"246,588.20",0.00,"2,294.80"
"57,719",0,0.00,0.00,507.00,0.00,3.80
"57,762",0,0.00,0.00,240.20,0.00,1.80
"57,774",0,0.00,0.00,988.60,0.00,8.20


In [23]:
total_deposit_shallow = read_and_cast("deposit_1").pipe(select_impuatable)
total_deposit_shallow.fetch().head()

case_id,num_group1,openingdate_313D,amount_416A
u32,u16,date,f32
"57,694",0,2016-10-12,"150,052.38"
"57,719",0,2016-09-22,368.39
"57,737",0,2016-05-13,202.00
"57,774",0,2016-03-03,733.01
"57,890",0,2015-03-12,"1,178.49"


In [24]:
total_debitcard_shallow = read_and_cast("debitcard").pipe(select_impuatable)
total_debitcard_shallow.fetch().head()

case_id,num_group1,openingdate_857D
u32,u16,date
"57,694",0,2016-10-12
"57,719",0,2016-09-22
"57,737",0,2016-05-13
"57,774",0,2016-03-03
"57,890",0,2015-03-12


In [25]:
total_credit_internal_shallow = read_and_cast("bureau_a_1").pipe(select_impuatable)
total_credit_internal_shallow.fetch().head()

case_id,num_group1,refreshdate_3813885D,classificationofcontr_13M,classificationofcontr_400M,contractst_545M,contractst_964M,description_351M,financialinstitution_382M,financialinstitution_591M,purposeofcred_426M,purposeofcred_874M,subjectrole_182M,subjectrole_93M
u32,u16,date,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat
"57,543",0,2020-10-17,"""ea6782cc""","""a55475b1""","""b919198c""","""a55475b1""","""a55475b1""","""a55475b1""","""b619fa46""","""60c73645""","""a55475b1""","""ab3c25cf""","""a55475b1"""
"57,543",1,2020-10-20,"""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1"""
"57,543",2,2020-10-19,"""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1"""
"57,543",3,null,"""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1"""
"57,543",4,2019-01-03,"""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1"""


In [26]:
total_credit_external_shallow = read_and_cast("bureau_b_1").pipe(select_impuatable)
total_credit_external_shallow.fetch().head()

case_id,num_group1,contractdate_551D,contractmaturitydate_151D,lastupdate_260D,classificationofcontr_1114M,contractst_516M,contracttype_653M,credor_3940957M,credquantity_1099L,credquantity_984L,dpdmaxdatemonth_804T,dpdmaxdateyear_742T,numberofinstls_810L,overdueamountmaxdatemonth_494T,overdueamountmaxdateyear_432T,periodicityofpmts_997M,pmtmethod_731M,pmtnumpending_403L,purposeofcred_722M,subjectrole_326M,subjectrole_43M,amount_1115A,debtpastduevalue_732A,debtvalue_227A,dpd_550P,dpd_733P,dpdmax_851P,installmentamount_644A,installmentamount_833A,instlamount_892A,maxdebtpduevalodued_3940955A,overdueamountmax_950A,pmtdaysoverdue_1135P,totalamount_503A,totalamount_881A
u32,u16,date,date,date,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32
"57,660",0,2019-12-08,2021-12-08,2020-10-10,"""ea6782cc""","""7241344e""","""1c9c5356""","""b619fa46""","""2.0""","""1.0""","""12.0""","""2019.0""",null,"""12.0""","""2019.0""","""a55475b1""","""a55475b1""",null,"""60c73645""","""ab3c25cf""","""ab3c25cf""",null,0.00,null,0.00,0.00,0.00,0.00,"109,099.58",null,0.00,0.00,0.00,"133,279.00","16,800.00"
"57,660",1,2013-03-14,2021-03-14,2020-10-20,"""ea6782cc""","""7241344e""","""1c9c5356""","""b619fa46""",null,null,"""10.0""","""2017.0""",null,"""8.0""","""2015.0""","""a55475b1""","""a55475b1""",null,"""60c73645""","""a55475b1""","""a55475b1""",null,0.00,null,null,null,"32,618.00",null,null,null,11.40,16.20,0.00,null,null
"57,679",0,2019-06-26,2021-06-26,2020-10-04,"""ea6782cc""","""7241344e""","""1c9c5356""","""b619fa46""","""1.0""","""2.0""","""9.0""","""2019.0""",null,"""9.0""","""2019.0""","""a55475b1""","""a55475b1""",null,"""60c73645""","""ab3c25cf""","""ab3c25cf""",null,0.00,null,0.00,0.00,"9,057.00",0.00,0.00,null,2.00,2.00,0.00,0.00,"86,800.00"
"57,679",1,2019-10-04,2021-10-04,2020-10-12,"""01f63ac8""","""7241344e""","""4257cbed""","""50babcd4""","""1.0""",null,"""10.0""","""2020.0""","""24.0""","""10.0""","""2020.0""","""a0b598e4""","""dbcbe8f8""","""12.0""","""60c73645""","""ab3c25cf""","""a55475b1""","60,000.00",0.00,"35,481.71",0.00,null,"17,310.00",null,"35,481.71","3,529.16",41.60,43.00,0.00,"60,000.00",null
"57,689",0,2019-12-31,2021-12-31,2020-10-04,"""ea6782cc""","""7241344e""","""1c9c5356""","""b619fa46""","""2.0""",null,"""2.0""","""2020.0""",null,"""2.0""","""2020.0""","""a55475b1""","""a55475b1""",null,"""60c73645""","""ab3c25cf""","""a55475b1""",null,0.00,null,0.00,null,0.00,null,"82,776.80",null,0.00,0.00,0.00,"144,958.80",null


In [27]:
total_credit_external_depth = read_and_cast("bureau_b_2").pipe(select_impuatable)
total_credit_external_depth.fetch().head()

case_id,num_group1,num_group2,pmts_date_1107D,pmts_dpdvalue_108P,pmts_pmtsoverdue_635A
u32,u16,u16,date,f32,f32
"57,660",1,16,2019-02-15,0.00,0.00
"57,660",1,30,2020-04-15,0.00,0.00
"57,660",1,34,2020-08-15,0.00,0.00
"57,660",1,26,2019-12-15,0.00,0.00
"57,660",0,9,2020-08-15,0.00,0.00


In [28]:
total_registry_a = read_and_cast("registry_a").pipe(select_impuatable)
total_registry_a.fetch().head()

case_id,num_group1,recorddate_4527225D,name_4527232M,amount_4527230A
u32,u16,date,cat,f32
"57,679",4,2020-10-20,"""ba006408""","1,120.00"
"57,679",3,2020-10-20,"""ba006408""","3,640.00"
"57,679",1,2020-10-20,"""ba006408""","3,122.00"
"57,679",0,2020-10-20,"""ba006408""","2,160.00"
"57,679",2,2020-10-20,"""ba006408""","1,494.60"


In [29]:
total_registry_b = read_and_cast("registry_b").pipe(select_impuatable)
total_registry_b.fetch().head()

case_id,num_group1,deductiondate_4917603D,name_4917606M,amount_4917619A
u32,u16,date,cat,f32
"57,543",1,2020-09-17,"""9f9384a2""","23,976.00"
"57,543",2,2020-09-17,"""9f9384a2""","23,310.20"
"57,543",4,2020-10-09,"""d2142f5c""","25,174.80"
"57,543",0,2020-07-03,"""d2142f5c""","23,976.00"
"57,543",3,2020-10-02,"""d2142f5c""","31,060.20"


In [30]:
total_registry_c = read_and_cast("registry_c").pipe(select_impuatable)
total_registry_c.fetch().head()

case_id,num_group1,processingdate_168D,employername_160M,pmtamount_36A
u32,u16,date,cat,f32
357,5,2018-08-08,"""c91b12ff""","1,100.00"
357,1,2018-11-28,"""c91b12ff""","1,200.00"
357,4,2018-09-10,"""c91b12ff""","1,200.00"
357,0,2019-01-04,"""c91b12ff""","1,200.00"
357,3,2018-10-08,"""c91b12ff""",628.60


In [31]:
try:
    for _ in globals().keys():
        if "total_" in _:
            print(_)
except RuntimeError:
    for _ in globals().keys():
        if "total_" in _:
            print(_)

total_base
total_past_shallow
total_past_depth
total_static_base
total_person_shallow
total_person_depth
total_other_shallow
total_deposit_shallow
total_debitcard_shallow
total_credit_internal_shallow
total_credit_external_shallow
total_credit_external_depth
total_registry_a
total_registry_b
total_registry_c


In [32]:
total_df = (
    total_base
    .join(
        total_past_shallow.drop(cs.contains("num_group")),
        left_on="case_id_base",
        right_on="case_id",
        how="left"
    )
    .join(
        total_past_depth.drop(cs.contains("num_group")),
        left_on="case_id_base",
        right_on="case_id",
        how="left"
    )
    .join(
        total_static_base.drop(cs.contains("num_group")),
        left_on="case_id_base",
        right_on="case_id",
        how="left"
    )
    .join(
        total_person_shallow.drop(cs.contains("num_group")),
        left_on="case_id_base",
        right_on="case_id",
        how="left"
    )
    .join(
        total_person_depth.drop(cs.contains("num_group")),
        left_on="case_id_base",
        right_on="case_id",
        how="left"
    )
    .join(
        total_other_shallow.drop(cs.contains("num_group")),
        left_on="case_id_base",
        right_on="case_id",
        how="left"
    )
    .join(
        total_deposit_shallow.drop(cs.contains("num_group")),
        left_on="case_id_base",
        right_on="case_id",
        how="left"
    )
    .join(
        total_debitcard_shallow.drop(cs.contains("num_group")),
        left_on="case_id_base",
        right_on="case_id",
        how="left"
    )
    .join(
        total_credit_internal_shallow.drop(cs.contains("num_group")),
        left_on="case_id_base",
        right_on="case_id",
        how="left"
    )
    .join(
        total_credit_external_shallow.drop(cs.contains("num_group")),
        left_on="case_id_base",
        right_on="case_id",
        how="left"
    )
    .join(
        total_credit_external_depth.drop(cs.contains("num_group")),
        left_on="case_id_base",
        right_on="case_id",
        how="left"
    )
    .join(
        total_registry_a.drop(cs.contains("num_group")),
        left_on="case_id_base",
        right_on="case_id",
        how="left"
    )
    .join(
        total_registry_b.drop(cs.contains("num_group")),
        left_on="case_id_base",
        right_on="case_id",
        how="left"
    )
    .join(
        total_registry_c.drop(cs.contains("num_group")),
        left_on="case_id_base",
        right_on="case_id",
        how="left"
    )
    .with_columns(
        (cs.date() - pl.col("Date"))
    )
)
total_df.collect(streaming=True)

: 